In [1]:
# This notebook analyzes the efficacy of different clustering algorithms for matching design patterns and design problems.
# Note: Table C2 in Hussain et al 2017 seems to indicate that fuzzy c-means clustering with binary weighting is the most effective combination for GoF patterns,
# but TF-IDF yields the highest f-value for fuzzy c-means (0.73).
# We are aiming for an f-value of 0.7 or above.

!pip install unidecode
!pip install fuzzy-c-means
!pip install --upgrade scikit-learn
!pip install scikit-learn-extra

from fcmeans                          import FCM

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data Structures
import numpy  as np
import pandas as pd
import json
# Corpus Processing
import re
import nltk
import nltk.corpus
from nltk.tokenize                    import word_tokenize
from nltk.stem                        import WordNetLemmatizer
from nltk                             import SnowballStemmer, PorterStemmer
from nltk.tag                         import pos_tag

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

from sklearn.feature_extraction.text  import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing            import normalize, Normalizer
from sklearn.decomposition            import PCA, TruncatedSVD
from sklearn.cluster                  import KMeans, BisectingKMeans, AgglomerativeClustering #, OPTICS
from sklearn_extra.cluster            import KMedoids
from sklearn.pipeline                 import make_pipeline

from unidecode                        import unidecode

# K-Means
from sklearn                          import cluster

# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
import seaborn            as sns
from sklearn.metrics                  import silhouette_samples, silhouette_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from wordcloud                        import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:

def getTrainData(fileName):
  df = pd.read_csv(fileName)
  df = df.drop_duplicates(subset=['name'])
  return df

In [3]:
# Add the design problem to the dataset.
problemRow = {'name':"bridge", 'correct_category':1, 'overview':"Design a system enabling to display on a screen some empty windows (no button, no menu…). A window can have several different styles depending on the platform used. We consider two platforms, XWindow and Presentation Manager. The client code must be written independently and without knowledge of the future execution platform. It is probable that the system evolves in order to display specialized windows by ‘application windows’ (able to manage applications) and ‘iconised windows’ (with an icon)"}
problemRow2 = {'name':"state", 'correct_category':0,  'overview':"Design a DVD market place work. The DVD marketplace provides DVD to its clients with three categories: children, normal and new. A DVD is new during some weeks, and after change category. The DVD price depends on the category. It is probable that the system evolves in order to take into account the horror category"}
problemRow3 = {'name':"mediator", 'correct_category':0,  'overview':"Design the communications of a plane approaching an airport. When a plane approaches an airport, it must announce to all the other planes which are around that it intends to land, and await their confirmation before carrying out the operation. It is the control tower of the airport which guarantees the regulation of the air traffic, by making sure that there is no trajectory or destination conflict between several planes. In addition to the class diagram, you must also submit a collaboration (in the form of a diagram of collaboration or a diagram of objects and sequence) that describes the landing of a plane amidst in a context of two demands to land and one wanting to take off"}
problemRow4 = {'name':"composite", 'correct_category':1, 'overview':"Design a system enabling you to draw a graphic image. A graphic image is composed of lines, rectangles, texts, and images. An image may be composed of other images, lines, rectangles, and texts"}
problemRow5 = {'name':"decorator", 'correct_category':1, 'overview':"Design a system enabling you to display visual objects on a screen. A visual object can be composed of one or more texts or images. If needed, the system must allow the addition of a vertical scroll bar, a horizontal scroll bar, an edge and a menu to this object. These additions may be accumulated."}
problemRow6 = {'name':"chain of responsibility", 'correct_category':0, 'overview':"Design a help manager for a Java application. A help manager allows the display of a help message depending on the objects on which a client has clicked. For example, the “?”, sometimes located near the contextual menu of a Windows dialog box, allows the display of the help related to the button or the area where to click. If the button on which one clicks does not contain help, it is the area container which displays its help, and so on. If any object contains help, the help manager displays /“No help available for this area/”. Instantiate your class diagram in a sequence diagram of the example of a printing window. This window (JDialog) consists in an explanatory text (JLabel) and in a container (JPanel). This last contains a /“Print button/” (JButton) and a /“Cancel button/” (JButton). The /“Print button/” contains help /“Launches the impression of the document/”. The /“Cancel button/” the text as well as the window do not contain help. Lastly, the container contains help /“Click on one of the buttons/”. In the sequence diagram, reveal the scenarios: /“The user asks for the help of the Print button/”, /“the user asks for the help of the Cancel button/”, and /“the user asks for the help of the text/”"}
problemRow7 = {'name':"command", 'correct_category':0, 'overview':"Design a tutorial to learn how to program a calculator. This calculator executes the four basic arithmetic operations. The goal of this tutorial is to make it possible to take a set of operations to be executed sequentially. The tutorial presents a button for each arithmetic operation, and two input fields for the operands. After each click on a button of an operation, the user has then the choice to start again or execute the sequence of operations to obtain the result. It is probable that this tutorial evolves in order to make it possible for the user to remove the last operation of the list and to take into account the operation of modulo"}
problemRow8 = {'name':"visitor", 'correct_category':0, 'overview':"Many distinct and unrelated operations need to be performed on node objects in a heterogeneous aggregate structure. You want to avoid ‘polluting’ the node classes with these operations. And, you don't want to have to query the type of each node and cast the pointer to the correct type before performing the desired operation."}
problemRow9 = {'name':"adapter", 'correct_category':1, 'overview':"Design a drawing editor. A design is composed of te graphics (lines, rectangles and roses), positioned at precise positions. Each graphic form must be modeled by a class that provides a method draw(): void. A rose is a complex graphic designed by a black-box class component. This component performs this drawing in memory, and provides access through a method getRose(): int that returns the address of the drawing. It is probable that the system evolves in order to draw circles"}
problemRow10 = {'name':"prototype", 'correct_category':2, 'overview':"Say you have an object, and you want to create an exact copy of it. How would you do it? First, you have to create a new object of the same class. Then you have to go through all the fields of the original object and copy their values over to the new object. Nice! But there’s a catch. Not all objects can be copied that way because some of the object’s fields may be private and not visible from outside of the object itself. What can go wrong when copying things from the outside? Copying an object from the outside isn’t always possible. There’s one more problem with the direct approach. Since you have to know the object’s class to create a duplicate, your code becomes dependent on that class. If the extra dependency doesn’t scare you, there’s another catch. Sometimes you only know the interface that the object follows, but not its concrete class, when, for example, a parameter in a method accepts any objects that follow some interface."}
problemRow11 = {'name':"factory method", 'correct_category':2, 'overview':"Imagine that you’re creating a logistics management application. The first version of your app can only handle transportation by trucks, so the bulk of your code lives inside the Truck class. After a while, your app becomes pretty popular. Each day you receive dozens of requests from sea transportation companies to incorporate sea logistics into the app. Adding a new transportation class to the program causes an issue Adding a new class to the program isn’t that simple if the rest of the code is already coupled to existing classes. Great news, right? But how about the code? At present, most of your code is coupled to the Truck class. Adding Ships into the app would require making changes to the entire codebase. Moreover, if later you decide to add another type of transportation to the app, you will probably need to make all of these changes again. As a result, you will end up with pretty nasty code, riddled with conditionals that switch the app’s behavior depending on the class of transportation objects."}
#TODO: we need some design problems related to creational patterns
#df = df.append(problemRow, ignore_index=True)
df_problems = pd.DataFrame()
df_problems = df_problems.append(problemRow, ignore_index=True)
df_problems = df_problems.append(problemRow2, ignore_index=True)
df_problems = df_problems.append(problemRow3, ignore_index=True)
df_problems = df_problems.append(problemRow4, ignore_index=True)
df_problems = df_problems.append(problemRow5, ignore_index=True)
df_problems = df_problems.append(problemRow6, ignore_index=True)
df_problems = df_problems.append(problemRow7, ignore_index=True)
df_problems = df_problems.append(problemRow8, ignore_index=True)
df_problems = df_problems.append(problemRow9, ignore_index=True)
df_problems = df_problems.append(problemRow10, ignore_index=True)
df_problems = df_problems.append(problemRow11, ignore_index=True)
df_problems

,name,correct_category,overview
0,bridge,1,Design a system enabling to display on a scree...
1,state,0,Design a DVD market place work. The DVD market...
2,mediator,0,Design the communications of a plane approachi...
3,composite,1,Design a system enabling you to draw a graphic...
4,decorator,1,Design a system enabling you to display visual...
5,chain of responsibility,0,Design a help manager for a Java application. ...
6,command,0,Design a tutorial to learn how to program a ca...
7,visitor,0,Many distinct and unrelated operations need to...
8,adapter,1,Design a drawing editor. A design is composed ...
9,prototype,2,"Say you have an object, and you want to create..."


In [4]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# applied lemmatization to a list of tokenized words
def applyLemmatization(listOfTokens, lemmatizer):
    return [lemmatizer.lemmatize(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

# removes any words that aren't verbs
def notVerbs(listOfTokens):
    notVerb = []
    for token in listOfTokens:
        if ( pos_tag(word_tokenize(token), tagset="universal")[0][1] != "VERB" and pos_tag(word_tokenize(token), tagset="universal")[0][1] != "ADJ" ):
            notVerb.append(token)
    return notVerb

In [5]:
def processCorpus(corpus, language, stemmer, lemmatizer):   
    stopwords = nltk.corpus.stopwords.words(language)
    param_stemmer = stemmer
    
    for document in corpus:
        index = corpus.index(document)
        corpus[index] = str(corpus[index]).replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        
        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)
        notVerb = notVerbs(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        #listOfTokens = removeWords(listOfTokens, notVerb)
        
        listOfTokens = applyStemming(listOfTokens, param_stemmer)
        #listOfTokens = applyLemmatization(listOfTokens, lemmatizer)

        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

In [6]:
# TODO: use k-means before this chunk of code to classify the problem with a pattern class, then perform cosine similarity with the problem and the list of candidate patterns from that class.
# Source: https://danielcaraway.github.io/html/sklearn_cosine_similarity.html

def cosine_sim(df, df_col, class_no, pos_to_last, predicted_labels):
  unigram_count = CountVectorizer(encoding='latin-1', binary=False)
  unigram_count_stop_remove = CountVectorizer(encoding='latin-1', binary=False, stop_words='english')

  # get the list of candidate patterns
  txts = df_col.loc[predicted_labels == class_no] # where label == class_no
  vecs = unigram_count.fit_transform(txts)
  
  cos_sim = cosine_similarity(vecs[-pos_to_last], vecs)
  #sim_sorted_doc_idx = cos_sim.argsort()
  # print the most similar pattern to the problem; it's actually the problem itself
  #print("Design Problem: \n" + txts.iloc[sim_sorted_doc_idx[-1][len(txts)-1]] + "\n")

  #bestFittingPatternDesc = txts.iloc[sim_sorted_doc_idx[-1][len(txts)-2]]

  # print the second most similar pattern; it's likely the best-fitting design pattern for the design problem
  #print(txts[sim_sorted_doc_idx[-1][len(txts)-2]])
  #print("\nCorrect Pattern: " + (df['name'][(df['overview'] == bestFittingPatternDesc)]).to_string(index=False) + "\n")

  return cos_sim, txts


In [7]:
def displayPredictions(cos_sim, txts, df):
  sim_sorted_doc_idx = cos_sim.argsort()
  for i in range(len(txts) - 1):
    patternDesc = txts.iloc[sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]
    patternName = (df['name'][(df['overview'] == patternDesc)]).to_string(index=False)
    percentMatch = int((cos_sim[0][sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]) * 100)
    print("{}th pattern:  {:<20}{}%  match".format(i+1, patternName, percentMatch))


In [8]:
def runAlgorithms(num_clusters, final_df, df):
  # bisecting_strategy{“biggest_inertia”, “largest_cluster”}, default=”biggest_inertia”
  final_df_array = final_df.to_numpy()

  Bi_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="biggest_inertia")
  Lc_Bisect = BisectingKMeans(n_clusters=num_clusters, bisecting_strategy="largest_cluster")
  Hierarchy = AgglomerativeClustering(n_clusters=num_clusters)
  Fuzzy_Means = FCM(n_clusters=num_clusters)
  Fuzzy_Means.fit(final_df_array)
  kmed = KMedoids(n_clusters=num_clusters)
  kmed_manhattan = KMedoids(n_clusters=num_clusters,metric='manhattan')
  Kmeans = cluster.KMeans(n_clusters = num_clusters)
  # optics = cluster.OPTICS(min_samples=num_clusters)

  Kmeans_labels = Kmeans.fit_predict(final_df)
  fuzzy_labels = Fuzzy_Means.predict(final_df_array)
  bi_bisect_labels = Bi_Bisect.fit_predict(final_df)
  lc_bisect_labels = Lc_Bisect.fit_predict(final_df)  
  hierarchy_labels = Hierarchy.fit_predict(final_df)
  kmed_labels = kmed.fit_predict(final_df)
  kmed_man_labels = kmed_manhattan.fit_predict(final_df)
  # optics_labels = optics.fit_predict(final_df_array)

  df['Kmeans'] = Kmeans_labels
  df['fuzzy'] = fuzzy_labels
  df['hierarchy'] = hierarchy_labels
  df['Bi_Bisect'] = bi_bisect_labels  
  df['Lc_Bisect'] = lc_bisect_labels
  df['PAM-EUCLIDEAN'] = kmed_labels
  df['PAM-MANHATTAN'] = kmed_man_labels
  # df['OPTICS'] = optics_labels


In [9]:
def Silhouette(vector_data, cluster_labels):
  # range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]
  #silhouette_avg = []
  # for num_clusters in range_n_clusters:
  #   # initialise kmeans
  #   kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
  #   kmeans.fit(vector_data)
  #   cluster_labels = kmeans.labels_
    
    # silhouette score
    #silhouette_avg.append(silhouette_score(vector_data, cluster_labels))
  s_avg = silhouette_score(vector_data, cluster_labels)
  return s_avg
  # plt.plot(range_n_clusters,silhouette_avg,'bx-')

  # plt.xlabel('Values of K') 
  # plt.ylabel('Silhouette score') 
  # plt.title('Silhouette analysis For Optimal k')
  # plt.show()

In [10]:
def getFScore(labels, df):
  df2 = df.pivot_table(index = ['correct_category'], aggfunc ='size')

  num_of_creational = df2[2]
  num_of_structural = df2[1]
  num_of_behavioral = df2[0]

  true_1 = [0]*num_of_creational + [1]*num_of_structural + [2]*num_of_behavioral
  true_2 = [0]*num_of_creational + [2]*num_of_structural + [1]*num_of_behavioral
  true_3 = [1]*num_of_creational + [0]*num_of_structural + [2]*num_of_behavioral
  true_4 = [1]*num_of_creational + [2]*num_of_structural + [0]*num_of_behavioral
  true_5 = [2]*num_of_creational + [0]*num_of_structural + [1]*num_of_behavioral
  true_6 = [2]*num_of_creational + [1]*num_of_structural + [0]*num_of_behavioral

  #print('===========KMEANS===========')
  #print('Predicted labels:')
  #display(Kmeans_labels.tolist())

  fscores = [
      f1_score(true_1, labels.tolist(),average='micro'),
      f1_score(true_2, labels.tolist(),average='micro'),
      f1_score(true_3, labels.tolist(),average='micro'),
      f1_score(true_4, labels.tolist(),average='micro'),
      f1_score(true_5, labels.tolist(),average='micro'),
      f1_score(true_6, labels.tolist(),average='micro')
  ]

  km_best = np.around(max(fscores),3)
  #print('\nBest fscore is:', km_best, 'from true_' + str(np.argmax(fscores) + 1))
  #display(globals()['true_' + str(np.argmax(fscores) + 1)])
  return km_best

In [11]:
def AdvancedTestDesignProblems():
  language = 'english'
  stemmer = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  vectorizer = TfidfVectorizer(sublinear_tf=True)
  df_test = getTrainData("GOF Patterns (2.0).csv")

  num_clusters = 3
  Fuzzy_Means = FCM(n_clusters=num_clusters, m=1.67, max_iter=270, random_state=23)

  while(True):
    for i in range(len(df_problems)):
      num_correct_prediction = 0

      # display design problem
      problemName = df_problems['name'].iloc[i]
      print("Design Problem ", i+1, ": ", problemName)
      
      # add design problem
      df_test = df_test.append(df_problems.iloc[i], ignore_index=True)

      # pre process
      corpus = df_test['overview'].tolist()
      corpus = processCorpus(corpus, language, stemmer, lemmatizer)

      # vectorize data
      X = vectorizer.fit_transform(corpus)
      tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

      final_df = tf_idf
      final_df_array = final_df.to_numpy()

      re_run = 20
      for j in range(re_run):
        # run model
        Fuzzy_Means.fit(final_df_array)
        fuzzy_labels = Fuzzy_Means.predict(final_df_array)
        df_test['Fmeans'] = fuzzy_labels

        labels = ["Fmeans"] 
        for i in range(len(labels)):
          print("\nRun number {} for {}:".format(j+1, labels[i]))
          # cosine similarity
          cos_sim, txts = cosine_sim(df_test, df_test['overview'], df_test[labels[i]].iloc[df_test.index[-1]], 1, df_test[labels[i]])
          #displayPredictions(cos_sim, txts, df_test)

          # get problem row
          n = df_test.index[-1]
          problemRow = df_test.iloc[[n]]

          sim_sorted_doc_idx = cos_sim.argsort()

          # loop patterns of the same cluster as the design problem
          for i in range(len(txts) - 1):
            # pattern predicted description
            patternDesc = txts.iloc[sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]
            # pattern name matching the description
            patternName = (df_test['name'][(df_test['overview'] == patternDesc)]).to_string(index=False)
            
            if patternName == problemName:
              num_correct_prediction += 1

            # percent cos sim match
            percentMatch = int((cos_sim[0][sim_sorted_doc_idx[-1][len(txts)-(i + 2)]]) * 100)
            print("{}th pattern:  {:<20}{}%  match".format(i+1, patternName, percentMatch))

      # clean up
      # Using drop() function to delete last row
      df_test.drop(index=n,axis=0,inplace=True)
      df_test.drop(['Fmeans'], axis=1)
      
      percent_correct = (num_correct_prediction/re_run) * 100
      print("{}% of time problem were correctly clustered. {} out of {}.\n\n".format(int(percent_correct), num_correct_prediction, re_run))

    return df_test, tf_idf, #cos_sim, txts 

#df, cos_sim, txts, tfidf = main()
df_test, tfidf = AdvancedTestDesignProblems()


Design Problem  1 :  bridge

Run number 1 for Fmeans:
1th pattern:  bridge              54%  match
2th pattern:  chain of responsibility54%  match
3th pattern:  command             53%  match
4th pattern:  memento             51%  match
5th pattern:  proxy               50%  match
6th pattern:  mediator            50%  match
7th pattern:  iterator            50%  match
8th pattern:  flyweight           49%  match
9th pattern:  singleton           46%  match

Run number 2 for Fmeans:
1th pattern:  bridge              54%  match
2th pattern:  chain of responsibility54%  match
3th pattern:  command             53%  match
4th pattern:  memento             51%  match
5th pattern:  proxy               50%  match
6th pattern:  mediator            50%  match
7th pattern:  iterator            50%  match
8th pattern:  flyweight           49%  match
9th pattern:  singleton           46%  match

Run number 3 for Fmeans:
1th pattern:  bridge              54%  match
2th pattern:  chain of responsibi